<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
<p></p>
</font>
<p></p>
<font>
<br>
Modern Information Retrieval course
<br>
lecturer: Dr. Soleymani
</font>
<p></p>
<br>
<font>
<b>
project third phase
</b>
</font>
<br>
<br>
<br>
<font>
<b>Amir Hossein Rahmati</b>
<br>
<font>
Sharif University of Technology
<br>
Computer Engineering department
<br>
<br>
</font>
</div>
</font>
</div>

<div>
<font size=5>
    <h1>
    <b>introduction</b>
    </h1>
    <p></p>
    <p></p>
</font>
<font size=3>
    <br>
    In this phase of the project, our focus will be on crawling and analyzing articles extracted from the Internet. We will start by examining various web crawling techniques to extract articles and other relevant information from the web.
    <br>
    In the next step, we will apply link analysis algorithms such as PageRank and HITS to determine the importance of these articles based on quotes, references, or other forms of links. We will also learn how to implement a personalized PageRank algorithm that takes user preferences into account to provide more relevant results.
    <br>
</font>
</div>
   

<div >
<font size=4>
    <h1>
    <b>
    Crawler Implementation
    </b>
    </h1>
</font>
    <br>

<font size=3>
   translate to English:
    In this section, you need to implement a Crawler to extract information from a number of articles from the <a href="https://www.semanticscholar.org/">Semantic Scholar</a> website.
    The extracted information must contain the following items.
</font>
</div>
<br>
<table dir="ltr" style="width: 100%; border-collapse: collapse;">
  <tr>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">ID</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">Title</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">Abstract</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">Publication Year</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">Authors</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">Related Topics</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">Citation Count</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">Reference Count</th>
    <th style="padding: 8px; text-align: justify; border: 1px solid black;">References</th>
  </tr>
  <tr>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">Unique ID of the paper</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">Title of the paper</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">Abstract of the paper</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">Publication year</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">Name of the first author, ..., Name of the last author</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">topic1, topic2, ...</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">number of citations of the paper</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">number of references of the paper</td>
    <td style="padding: 8px; text-align: justify; border: 1px solid black;">ID of the first reference, ..., ID of the tenth reference</td>
  </tr>
</table>
    <br>

<div>
<font size=3>
    translate to English:
    Start the extraction process from 5 articles of each professor and add the first 10 references of each article to the article queue.
    Continue the extraction process until you have information on 2000 articles.
    Save the article information in the crawled_paper_profName.json file.

</font>
</div>

<div >

<font size=3>
In implementing the Crawler, pay attention to the following.
<ul>
<li>You do not have the right to use the semantic scholar site api.</li>
<li>For crawling, you can use packages such as <a href="https://www.selenium.dev/selenium/docs/api/py/">Selenium</a> or <a href="https://gihub.com/scrapy/scrapy">Scrapy</a>. The use of other packages is also allowed. You can also use the <a href="https://pypi.org/project/beautifulsoup4/">Beautiful Soup</a> package to parse the extracted information.</li>
<li>Give a few seconds of delay between each request to the site.</li>
<li>At the time of delivery, your Crawler will be executed and its correctness will be checked.</li>
<li>If your Crawler encounters an error such as request timeout, it should not stop its work.</li>
</ul>

</font>
</div>

In [1]:
import requests
def get_html(url):
    payload = {'api_key': 'c30935129c107b958a0ff0096be3994a' , 'url' : url}
    return requests.get('http://api.scraperapi.com', params=payload)

In [35]:
from bs4 import BeautifulSoup
import re
def get_data(explored_urls, urls, url, html):
  ID = url.split("/")[-1]

  # title , year , authors
  doc = BeautifulSoup(html.text, 'html.parser')
  text = doc.find('pre', {'class': 'bibtex-citation'}).text
  title = re.search('title={(.+)}', text).group(1)
  year = int(re.search('year={(.+)}', text).group(1))
  authors = " , ".join(re.search('author={(.+)}', text).group(1).split(' and '))
  # abstract
  try:
      abstract = doc.find('span', {'data-test-id': 'text-truncator-text'}).string
  except:
      abstract = ""

  # related_topics
  try:
      related_topics = doc.find_all('li', {'class':'paper-meta-item'})[2].string
  except:
      related_topics = ""

  # citation_count and references_count
  counts_data = doc.findAll('h2', {'class': 'dropdown-filters__result-count__header dropdown-filters__result-count__citations'})
  citations_count = 0
  references_count = 0
  for data in counts_data:
    try:
      data_splits = data.string.split()
      if data_splits[1] == "Citations":
          citations_count = int(data_splits[0])
      elif data_splits[1] == "References":
        references_count = int(data_splits[0])
    except:
      pass

  # references
    references = doc.findAll(
      'a', {
          'data-test-id': 'citation-paper-title',
          'data-heap-citation-type': 'citedPapers',
      })

    for reference in references:
        if ('https://www.semanticscholar.org' + reference['href']) not in explored_urls:
          if ('https://www.semanticscholar.org' + reference['href']) not in urls:
            urls.append('https://www.semanticscholar.org' + reference['href'])

    paper = {'ID': ID, 'Title': title, 'Abstract': abstract, 'Publication Year': year, 'Authors': authors, 'Related Topics': related_topics
               , 'Citation Count': citations_count, 'Reference Count':references_count, 'References':[r['data-heap-paper-id'] for r in references]}

    return paper

In [36]:
prof_names = ["Kasaei", "Rabiee", "Rohban", "Sharifi", "Soleymani"]
URLS = {}
for prof_name in prof_names:
  with open(prof_name + ".txt") as file:
      URLS[prof_name] = file.readlines()

In [37]:
import json
from tqdm import tqdm
for prof_name in prof_names:
    crawled_papers = []
    urls = URLS[prof_name]
    explored_urls = []
    for i in tqdm(range(25)):
      if len(urls) == 0:
        break
      url = urls.pop(0)
      html = get_html(url)
      paper = get_data(explored_urls, urls, url, html)
      crawled_papers.append(paper)
      explored_urls.append(url)
    with open('crawled_paper_' + prof_name + '.json', 'w') as j:
        json.dump(crawled_papers, j)





100%|██████████| 25/25 [03:36<00:00,  8.67s/it]


<div >
<font  size=4>
    <h1>
    <b>
    personalized PageRank
    </b>
    </h1>
</font>
    <br>
<font size=3>
In this section, we implement the personalized PageRank algorithm, an extension of the PageRank algorithm that takes user preferences into account. The personalized PageRank algorithm ranks nodes in a graph based on their importance to the user, not based on their overall importance in the graph.

</font>
</div>

In [100]:
import numpy as np
from typing import List , Dict
def compute_p(graph):
    p = np.zeros((len(graph),len(graph)))
    for i in range(len(graph)):
      ref_n = len(graph[list(graph.keys())[i]])
      if ref_n == 0 :
        p[i] = np.ones(len(graph)) / len(graph)
      else:
        p[i] = np.array(list(map(lambda x: 1 if x else 0, [j in graph[list(graph.keys())[i]] for j in graph.keys()]))) / ref_n

    return p

def find_steady_a(a,p):
    dif = 1
    while dif > 0.000000001:
      a_hat = np.dot(a,p)
      dif = np.linalg.norm(a_hat - a)
      a = a_hat

    return a_hat


def pagerank(graph: Dict[str, List[str]]) -> Dict[str, float]:
    """
    Returns the personalized PageRank scores for the nodes in the graph, given the user's preferences.

    Parameters:
    graph (Dict[str, List[str]]): The graph represented as a dictionary of node IDs and their outgoing edges.

    Returns:
    Dict[str, float]: A dictionary of node IDs and their personalized PageRank scores.
    """

    a = np.zeros(len(graph))
    a[0:5] = 1/5
    p = compute_p(graph)
    a = find_steady_a(a,p)
    return dict(zip(list(graph.keys()) , a))


<div >
<font size=3>
In this section, we use the personalized PageRank algorithm implemented in the previous section to identify important articles related to a specific professor's field of work. This function takes a field as input and outputs the top articles that have the most connections to that field.
</font>
</div>

In [101]:
def important_articles(Professor: str) -> List[str]:
    """
    Returns the most important articles in the field of given professor, based on the personalized PageRank scores.

    Parameters:
    Professor (str): Professor's name.

    Returns:
    List[str]: A list of article IDs representing the most important articles in the field of given professor.
    """
    graph = {}
    with open(f'crawled_paper_{Professor}.json','r') as f:
      crawled_papers = json.load(f)
    for paper in crawled_papers:
      graph[paper['ID']] = paper['References']
      for ref in paper['References']:
        graph.setdefault(ref, [])

    im_papers = sorted(pagerank(graph).items(), key = lambda x: x[1], reverse=True)[:10]
    return im_papers

In [103]:
im_papers = important_articles("Kasaei")
for i in im_papers:
  print("paper_id = " + i[0] + " score = " + str(i[1]))

paper_id = 4b1a47709d0546e5bc614bf9a521c550e6881d04 score = 0.00529375052118805
paper_id = 505f48d8236eb25f871da272c2ac2fe4b41ea289 score = 0.005206013837088638
paper_id = eda3368a5198ca55768b07b6f5667aea28baf2cd score = 0.005107287548373918
paper_id = 201631fbc3f7d7cb2c1ddfaf82278cad5e44f2f9 score = 0.004370070110121608
paper_id = b7d540cd0de72e984cdec44afa4a4d039cfd5eea score = 0.004301336099850455
paper_id = bfba194dfd9c7c27683082aa8331adc4c5963a0d score = 0.003690278561280304
paper_id = 9926020dda21874dc7a5ef1511bae6c4cef5ecb9 score = 0.0036640691401314874
paper_id = 6767812e114c426d45ea83894b156f7906e525cd score = 0.0033284967523998037
paper_id = c63a34ac6a4e049118070e707ca7679fbb132d33 score = 0.003275420181468577
paper_id = 966aad492f75b17f698e981e008b73b51816c6aa score = 0.0031814144385978135


<div  style="text-align: justify">
<font  size=4>
    <h1>
    <b>
    autohrs ranking
    </b>
    </h1>
</font>
    <br>
<font size=3>  
The concept of authors referring to each other is used to rank authors. When author A refers to article P, which author B is one of the authors of that article, i.e., article P, we say that author A has referred to author B. With this relationship, we can create a graph of references between authors and then use the HITS algorithm to rank authors. To rank, we need to use hub and authority indices.

</font>
</div>

In [107]:
import networkx as nx
import networkx as nx


def hit_algorithm(papers, n):
    """
        Implementing the HITS algorithm to score authors based on their papers and co-authors.

        Parameters
        ---------------------------------------------------------------------------------------------------
        papers: A list of paper dictionaries with the following keys:
                "id": A unique ID for the paper
                "title": The title of the paper
                "abstract": The abstract of the paper
                "date": The year in which the paper was published
                "authors": A list of the names of the authors of the paper
                "related_topics": A list of IDs for related topics (optional)
                "citation_count": The number of times the paper has been cited (optional)
                "reference_count": The number of references in the paper (optional)
                "references": A list of IDs for papers that are cited in the paper (optional)
        n: An integer representing the number of top authors to return.

        Returns
        ---------------------------------------------------------------------------------------------------
        List
        list of the top n authors based on their hub scores.
    """
    # Create a graph of authors and papers (all of the authors and papers represented as nodes, and all of the authors who wrote each paper connected to the corresponding paper node by an edge)
    G = nx.DiGraph()

    # Run the HITS algorithm
    hubs, authorities = nx.hits(G)

    # Create a list of top n authors based on their hub scores

    for paper in papers:
      authors = paper['Authors']
      for author in authors:
        G.add_edge(author, paper['ID'])

    hubs, authorities = nx.hits(G)

    top_authors = sorted(hubs.items(), key=lambda x: x[1], reverse=True)[:n]
    return top_authors

# call the hit_algorithm function
with open('crawled_paper_' + prof_name + '.json') as f:
  papers = json.load(f)
  print(hit_algorithm(papers, 10))


[('B. Mirheidari', 0.33333333333333476), ('H. Christensen', 0.33333333333333476), ('Yilin Pan', 0.23385403957214246), ('D. Blackburn', 0.09947929376119231), ('S. Orimaye', 2.862767636958744e-15), ('Jojo Sze-Meng Wong', 2.3707264061142116e-15), ('Judyanne Sharmini Gilbert Fernandez', 7.947068100655231e-16), ('Ireneous N. Soyiri', 7.947068100655231e-16), ('Karen J. Golden', 7.813127859831653e-16), ('Zeynep Akata', 6.107033129827701e-16)]
